# Heron Bench

目的：Heron Benchに親しむ

In [6]:
from datasets import load_dataset

dataset_name = "turing-motors/Japanese-Heron-Bench"
dataset = load_dataset(dataset_name)

In [7]:
dataset['train'][0]

{'image': <PIL.Image.Image image mode=RGB size=1920x1038>}

In [8]:
for i, sample in enumerate(dataset['train']):
    idx = i + 1
    # format to xxx.png
    idx = str(idx).zfill(3)
    sample['image'].save(f"tmp/{idx}.jpg")

[事実]
- データセットには画像のみ格納されている
- したがって，それらをjoinして扱いやすいデータセットを作成した上でllm-jpにアップロードする

In [9]:
import pandas as pd
from datasets import Dataset, Image

df = pd.read_json('../data/japanese-heron-bench/questions_ja.jsonl', lines=True)
df['image'] = df['image'].apply(lambda x: f'tmp/{x}')
heron_bench_llmjp_hf = Dataset.from_pandas(df)
heron_bench_llmjp_hf = heron_bench_llmjp_hf.cast_column('image', Image())

In [11]:
from huggingface_hub import HfApi, HfFolder

# データセットをHugging Face Hubにプッシュする関数
def push_dataset_to_hub(dataset, repo_id):
    # データセットのプッシュ
    dataset.push_to_hub(repo_id)

# HfApiインスタンスを作成
api = HfApi()

# HfFolderからトークンを取得
token = HfFolder.get_token()

# データセットをプッシュするリポジトリID
repo_id = "Silviase/Japanese-Heron-Bench"

# データセットをHugging Face Hubにプッシュ
push_dataset_to_hub(heron_bench_llmjp_hf, repo_id)


Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


In [16]:
dataset_from_hf = load_dataset("Silviase/Japanese-Heron-Bench")
dataset_from_hf['train'][0]

{'question_id': 0,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1038>,
 'category': 'complex',
 'image_category': 'anime',
 'context': '宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽が置かれており、少年が小型ナイフでそのうちの一つを加工しています。他には竹の棒や赤いワイヤー、２本の鉛筆が机の左側に置かれており、右側には数冊の本といくつかの書類が置かれている。',
 'text': 'この作品は著名な映画監督により制作されました。イラストから誰の作品だと考えられますか？'}